# Output Validations

<a href="https://colab.research.google.com/github/stevengonsalvez/LLM-dojo/blob/main/gpt-api/HLW-6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
!pip install openai tiktoken python-dotenv

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

### Execute when running in colab with env var in drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import os

!source /content/drive/MyDrive/env_vars.sh

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### Check output for potentially harmful content

In [ ]:
final_response_to_customer = f"""
The iPhone 14 Pro is the latest flagship phone from Apple. It features a 6.7-inch Super Retina XDR display, an A15 Bionic chip, and 128GB of storage. It comes with a 1-year warranty and is priced at £70 per month. Can I help you with anything else about the iPhone 14 Pro?
The Nintendo Switch is a versatile gaming console that allows you to play games both on your TV and as a handheld device. It has a wide library of games and comes with a 1-year warranty. The Nintendo Switch is priced at £30 per month. Is there anything specific you would like to know about the Nintendo Switch?
Our broadband service offers superfast fibre optic internet with unlimited data and high download and upload speeds. It is priced at £35 per month. Can I assist you with any other questions about our broadband service?
"""
response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

### Check if output relevance and accuracy

In [ ]:
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""
tell me about the iphone 14 pro phone and \
 the nintendo switch. \
 Also tell me about your broadband """
product_information = """[{"name":"iPhone 14 Pro","category":"Phones","brand":"Apple","model_number":"14 Pro","warranty":"1 year","rating":4.5,"features":["6.7-inch Super Retina XDR display","A15 Bionic chip","128GB storage"],"description":"The latest flagship iPhone with a stunning display and powerful performance.","price_monthly":"£70"},{"name":"Nintendo Switch","category":"Accessories and Electronics","brand":"Nintendo","model_number":"Switch","warranty":"1 year","rating":4.5,"features":["Hybrid gaming console","Play at home or on the go","Wide library of games"],"description":"Versatile gaming console that allows you to play your favorite games both on your TV and as a handheld device.","price_monthly":"£30"},{"name":"Fibre Broadband","category":"Broadband","brand":"Various","model_number":"N/A","warranty":"1 year","rating":4.5,"features":["Superfast fibre optic broadband","Unlimited data","High download and upload speeds"],"description":"High-speed fibre broadband service offering fast and stable internet connection for your home.","price_monthly":"£35"}]"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

> Chain of thought reasoning on the output validation
> - example: validation of tone against brand guidelines (with some specific outlines on brand guidelines )
> - could have a set of guidelines/rules against which this validation is done, based on type of user or context

In [ ]:
another_response = "May the force be with you"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)